In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from camp2ascii import camp2ascii


In [ ]:
in_dir = Path("tests/raw")

out_dir = Path("tests/c2a-basic")
out_dir.mkdir(parents=True, exist_ok=True)
for f in out_dir.iterdir():
    f.unlink() if f.is_file() else None
out_files = camp2ascii(in_dir, out_dir)

my_tob3 = pd.read_csv(out_files[2], skiprows=[0, 2, 3], parse_dates=["TIMESTAMP"], index_col="TIMESTAMP", na_values="NAN")
ref_tob3 = pd.read_csv(Path("tests/cc-basic/TOA5_TOB3_ring2.dat"), skiprows=[0, 2, 3], parse_dates=["TIMESTAMP"], index_col="TIMESTAMP", na_values="NAN")
my_tob3.shape, ref_tob3.shape

my_tob3

,RECORD,temp(1),temp(2),temp(3),temp(4),temp(5),text_val,toggle,temp_bool8(1),temp_bool8(2),temp(8)
TIMESTAMP,,,,,,,,,,,
2026-02-18 11:38:10.200,100,22.86,0.0,25.150785,4050,405000,405000,1,10110,10110,0
2026-02-18 11:38:13.200,115,22.87,0.0,25.153639,4650,465000,465000,1,10110,10110,0
2026-02-18 11:38:16.200,130,NaN,0.0,25.156290,5250,525000,525000,1,0,0,0
2026-02-18 11:41:30.200,101,22.86,0.0,25.150986,4090,409000,409000,1,10110,10110,0
2026-02-18 11:41:33.200,116,NaN,0.0,25.153772,4690,469000,469000,1,0,0,0
2026-02-18 11:41:36.200,131,22.87,0.0,25.156424,5290,529000,529000,1,10110,10110,0
2026-02-18 11:44:50.200,102,NaN,0.0,25.151188,4130,413000,413000,1,0,0,0
2026-02-18 11:44:53.200,117,22.87,0.0,25.154007,4730,473000,473000,1,10110,10110,0
2026-02-18 11:44:56.200,132,22.87,0.0,25.156591,5330,533000,533000,1,10110,10110,0


In [ ]:

out_dir = Path("tests/c2a-timedate-filenames")
out_dir.mkdir(parents=True, exist_ok=True)
for f in out_dir.iterdir():
    f.unlink() if f.is_file() else None
camp2ascii(in_dir, out_dir, timedate_filenames=2)

in_dir = Path("tests/raw-2")
out_dir = Path("tests/c2a-time-split-1")
out_dir.mkdir(parents=True, exist_ok=True)
for f in out_dir.iterdir():
    f.unlink() if f.is_file() else None
camp2ascii(in_dir, out_dir, time_interval="1m")

out_dir = Path("tests/c2a-time-split-2")
out_dir.mkdir(parents=True, exist_ok=True)
for f in out_dir.iterdir():
    f.unlink() if f.is_file() else None
camp2ascii(in_dir, out_dir, time_interval="3m", timedate_filenames=1, contiguous_timeseries=2)